Importações(tudo certo)

In [30]:
import psycopg2 #pip install psycopg2 ou pip install psycopg2-binary
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import shapiro, kruskal, f_oneway
import statsmodels.api as sm
import matplotlib.pyplot as plt

Conexão com o banco de Dados(tudo certo)

In [31]:
import psycopg2
import pandas as pd

# Parâmetros de conexão
host = "localhost"         # Substitua pelo IP/hostname do servidor, se não for local
port = "5432"              # Porta padrão do PostgreSQL
database = "Projeto"       # Nome do banco de dados
user = "postgres"          # Nome do usuário
password = "teste"         # Senha do usuário

# Conectar ao banco de dados
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    print("Conexão bem-sucedida!")

    # Criar um cursor para executar comandos SQL
    cursor = connection.cursor()

    # Carregar dados diretamente do banco para o pandas
    query = """
    SELECT 
        r.algoritmo,
        r.cenario,
        r.tempoexecucao,
        m.processador,
        m.ram
    FROM resultados r
    JOIN maquina m ON r.id_maquina_a = m.id_maquina;
    """
    df = pd.read_sql_query(query, connection)
    print("Dados carregados com sucesso!")
    print(df.head())  # Mostrar as 5 primeiras linhas

except psycopg2.Error as e:
    print("Erro ao conectar ao PostgreSQL:", e)

# Fechar a conexão e o cursor
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexão fechada.")

Conexão bem-sucedida!
Dados carregados com sucesso!
     algoritmo                               cenario  tempoexecucao  \
0  Bubble Sort  Cenário 1 - Vetor ordenado crescente         595886   
1   Merge Sort  Cenário 1 - Vetor ordenado crescente           6472   
2   Quick Sort  Cenário 1 - Vetor ordenado crescente           4498   
3   Shell Sort  Cenário 1 - Vetor ordenado crescente           6782   
4  Bubble Sort  Cenário 1 - Vetor ordenado crescente         557253   

           processador   ram  
0  intel core i5 10gen  16GB  
1  intel core i5 10gen  16GB  
2  intel core i5 10gen  16GB  
3  intel core i5 10gen  16GB  
4  intel core i5 10gen  16GB  
Conexão fechada.


C:\Users\lucas\AppData\Local\Temp\ipykernel_24668\626593929.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


analise inicial(certo)

In [32]:
# Verificar as primeiras linhas do DataFrame para entender a estrutura
print(df.head())

     algoritmo                               cenario  tempoexecucao  \
0  Bubble Sort  Cenário 1 - Vetor ordenado crescente         595886   
1   Merge Sort  Cenário 1 - Vetor ordenado crescente           6472   
2   Quick Sort  Cenário 1 - Vetor ordenado crescente           4498   
3   Shell Sort  Cenário 1 - Vetor ordenado crescente           6782   
4  Bubble Sort  Cenário 1 - Vetor ordenado crescente         557253   

           processador   ram  
0  intel core i5 10gen  16GB  
1  intel core i5 10gen  16GB  
2  intel core i5 10gen  16GB  
3  intel core i5 10gen  16GB  
4  intel core i5 10gen  16GB  


certo

In [33]:
# Estatísticas gerais de tempo de execução
print("\nEstatísticas gerais (tempo de execução):")
print(df['tempoexecucao'].describe())


Estatísticas gerais (tempo de execução):
count    2.160000e+03
mean     9.866122e+07
std      5.553921e+08
min      1.178000e+03
25%      2.316750e+03
50%      2.381350e+04
75%      3.817215e+05
max      4.586536e+09
Name: tempoexecucao, dtype: float64


certo

In [34]:
# Estatísticas por Algoritmo (média, desvio padrão, mínimo, máximo, etc.)
print("\nEstatísticas por Algoritmo:")
algoritmo_stats = df.groupby("algoritmo")["tempoexecucao"].agg(['mean', 'std', 'min', 'max', 'count'])
print(algoritmo_stats)


Estatísticas por Algoritmo:
                     mean           std     min         max  count
algoritmo                                                         
Bubble Sort  3.944837e+08  1.057668e+09  370047  4586535883    540
Merge Sort   3.118237e+04  5.612014e+04    1408      449850    540
Quick Sort   3.170331e+04  6.016756e+04    1178      300530    540
Shell Sort   9.828180e+04  2.092653e+05    2092      984104    540


certo

In [35]:
# Estatísticas por Cenário (tamanho da amostra) e Algoritmo
print("\nEstatísticas por Cenário e Algoritmo:")
cenario_stats = df.groupby(["cenario", "algoritmo"])["tempoexecucao"].agg(['mean', 'std', 'min', 'max', 'count'])
print(cenario_stats)


Estatísticas por Cenário e Algoritmo:
                                                             mean  \
cenario                                 algoritmo                   
Cenário 1 - Vetor ordenado crescente    Bubble Sort  4.162977e+08   
                                        Merge Sort   3.152770e+04   
                                        Quick Sort   3.330232e+04   
                                        Shell Sort   1.041741e+05   
Cenário 2 - Vetor ordenado decrescente  Bubble Sort  3.738545e+08   
                                        Merge Sort   3.073189e+04   
                                        Quick Sort   3.163922e+04   
                                        Shell Sort   9.941184e+04   
Cenário 3 - Vetor aleatório             Bubble Sort  3.798978e+08   
                                        Merge Sort   3.140327e+04   
                                        Quick Sort   3.107602e+04   
                                        Shell Sort   9.524615e+0

certo

In [36]:
# Estatísticas por Processador (usando a variável de RAM para o tamanho de amostra)
print("\nEstatísticas por Processador e RAM:")
processador_ram_stats = df.groupby(["processador", "ram"])["tempoexecucao"].agg(['mean', 'std', 'min', 'max', 'count'])
print(processador_ram_stats)


Estatísticas por Processador e RAM:
                                   mean           std   min         max  count
processador          ram                                                      
intel core i5 10gen  16GB  1.328563e+08  6.625409e+08  1530  4586535883   1120
intel core i5 12500h 8GB   6.183576e+07  4.069290e+08  1178  3931355592   1040


Verificar normalidade dos tempos de execução por algoritmo:

In [37]:
# Agrupar os tempos de execução por algoritmo
algoritmos = df['algoritmo'].unique()

print("Testes de Normalidade (Shapiro-Wilk):")
for algoritmo in algoritmos:
    tempos = df[df['algoritmo'] == algoritmo]['tempoexecucao']
    stat, p = shapiro(tempos)
    print(f"Algoritmo: {algoritmo}, p-valor: {p:.4f}")
    if p > 0.05:
        print(f"  -> Os dados seguem uma distribuição normal (p > 0.05).")
    else:
        print(f"  -> Os dados NÃO seguem uma distribuição normal (p <= 0.05).")

Testes de Normalidade (Shapiro-Wilk):
Algoritmo: Bubble Sort, p-valor: 0.0000
  -> Os dados NÃO seguem uma distribuição normal (p <= 0.05).
Algoritmo: Merge Sort, p-valor: 0.0000
  -> Os dados NÃO seguem uma distribuição normal (p <= 0.05).
Algoritmo: Quick Sort, p-valor: 0.0000
  -> Os dados NÃO seguem uma distribuição normal (p <= 0.05).
Algoritmo: Shell Sort, p-valor: 0.0000
  -> Os dados NÃO seguem uma distribuição normal (p <= 0.05).


Comparar médias entre os algoritmos

In [38]:
# Tempos por algoritmo
tempos_algoritmos = [df[df['algoritmo'] == algoritmo]['tempoexecucao'] for algoritmo in algoritmos]

# Teste ANOVA
anova_stat, anova_p = f_oneway(*tempos_algoritmos)
print("\nTeste ANOVA:")
print(f"Estatística F: {anova_stat:.4f}, p-valor: {anova_p:.4f}")

if anova_p > 0.05:
    print("  -> Não há diferenças significativas entre os algoritmos (p > 0.05).")
else:
    print("  -> Há diferenças significativas entre os algoritmos (p <= 0.05).")



Teste ANOVA:
Estatística F: 75.0992, p-valor: 0.0000
  -> Há diferenças significativas entre os algoritmos (p <= 0.05).


In [39]:
# Teste Kruskal-Wallis
kruskal_stat, kruskal_p = kruskal(*tempos_algoritmos)
print("\nTeste Kruskal-Wallis:")
print(f"Estatística H: {kruskal_stat:.4f}, p-valor: {kruskal_p:.4f}")

if kruskal_p > 0.05:
    print("  -> Não há diferenças significativas entre os algoritmos (p > 0.05).")
else:
    print("  -> Há diferenças significativas entre os algoritmos (p <= 0.05).")


Teste Kruskal-Wallis:
Estatística H: 1260.0549, p-valor: 0.0000
  -> Há diferenças significativas entre os algoritmos (p <= 0.05).
